<a href="https://colab.research.google.com/github/mondtorsha/ASR-Frameworks/blob/main/NvidiaNemo_ASR_using_with_MIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ffmpeg-python

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
#record the audio
audio, sr = get_audio()

In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

In [ ]:
BRANCH = 'stick-torch-171'
!python -m pip install git+https://github.com/tiborkiss/NeMo.git@$BRANCH#egg=nemo_toolkit[all]


  Cloning https://github.com/tiborkiss/NeMo.git (to revision stick-torch-171) to /tmp/pip-install-oac5lsd5/nemo-toolkit
  Running command git clone -q https://github.com/tiborkiss/NeMo.git /tmp/pip-install-oac5lsd5/nemo-toolkit
  Running command git checkout -b stick-torch-171 --track origin/stick-torch-171
  Switched to a new branch 'stick-torch-171'
  Branch 'stick-torch-171' set up to track remote branch 'stick-torch-171' from 'origin'.
     |████████████████████████████████| 14.5MB 249kB/s 
     |████████████████████████████████| 686kB 56.1MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 112kB 57.6MB/s 
     |████████████████████████████████| 133kB 55.2MB/s 
     |████████████████████████████████| 1.9MB 54.5MB/s 
     |████████████████████████████████| 1.2MB 52.0MB/s 
     |████████████████████████████████| 768kB 47.6MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 51kB 8.0MB/s

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import nemo
# Import Speech Recognition collection
import nemo.collections.asr as nemo_asr
# Import Natural Language Processing colleciton
import nemo.collections.nlp as nemo_nlp
# Import Speech Synthesis collection
import nemo.collections.tts as nemo_tts
# We'll use this to listen to audio
import IPython

[NeMo W 2021-03-12 15:56:29 experimental:28] Module <class 'nemo.collections.asr.models.clustering_diarizer.ClusteringDiarizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-03-12 15:56:29 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
Audio_sample= "recording.wav"
IPython.display.Audio(Audio_sample)

In [ ]:
# Speech Recognition model - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En").cuda()
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='Punctuation_Capitalization_with_DistilBERT').cuda()
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="Tacotron2-22050Hz").cuda()
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="WaveGlow-22050Hz").cuda()

[NeMo I 2021-03-12 15:56:44 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0rc1/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo
[NeMo I 2021-03-12 15:56:48 common:615] Instantiating model from pre-trained checkpoint
[NeMo I 2021-03-12 15:56:49 features:240] PADDING: 16
[NeMo I 2021-03-12 15:56:49 features:256] STFT using torch
[NeMo I 2021-03-12 15:56:52 modelPT:376] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.
[NeMo I 2021-03-12 15:56:52 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemonlpmodels/versions/1.0.0a5/files/Punctuation_Capitalization_with_DistilBERT.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0rc1/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d2

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


[NeMo W 2021-03-12 15:57:03 modelPT:133] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-03-12 15:57:03 modelPT:140] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    ds_item: null
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-03-12 15:57:03 modelPT:1134] World size can only be set by PyTorch Lightning Trainer.



[NeMo I 2021-03-12 15:57:11 modelPT:376] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/Punctuation_Capitalization_with_DistilBERT.nemo.
[NeMo I 2021-03-12 15:57:11 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemottsmodels/versions/1.0.0a5/files/Tacotron2-22050Hz.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0rc1/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo
[NeMo I 2021-03-12 15:57:15 common:615] Instantiating model from pre-trained checkpoint
[NeMo I 2021-03-12 15:57:16 features:240] PADDING: 16
[NeMo I 2021-03-12 15:57:16 features:249] STFT using conv
[NeMo I 2021-03-12 15:57:18 modelPT:376] Model Tacotron2Model was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo.
[NeMo I 2021-03-12 15:57:18 cloud:66] Download

In [ ]:
# Convert our audio sample to text
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  raw_text = transcription

# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]
print(f'\nRaw recognized text: {raw_text}. \nText with capitalization and punctuation: {text}')

[NeMo W 2021-03-12 15:58:10 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.



[NeMo I 2021-03-12 15:58:10 punctuation_capitalization_model:385] Using batch size 1 for inference
[NeMo I 2021-03-12 15:58:10 punctuation_capitalization_dataset:134] Max length: 12
[NeMo I 2021-03-12 15:58:10 data_preprocessing:297] Some stats of the lengths of the sequences:
[NeMo I 2021-03-12 15:58:10 data_preprocessing:303] Min: 12 |                  Max: 12 |                  Mean: 12.0 |                  Median: 12.0
[NeMo I 2021-03-12 15:58:10 data_preprocessing:305] 75 percentile: 12.00
[NeMo I 2021-03-12 15:58:10 data_preprocessing:306] 99 percentile: 12.00
[NeMo I 2021-03-12 15:58:10 punctuation_capitalization_dataset:165] 0 are longer than 12
[NeMo I 2021-03-12 15:58:10 punctuation_capitalization_dataset:168] *** Example ***
[NeMo I 2021-03-12 15:58:10 punctuation_capitalization_dataset:169] i: 0
[NeMo I 2021-03-12 15:58:10 punctuation_capitalization_dataset:170] subtokens: [CLS] hello hello hello how are you hello how are you [SEP]
[NeMo I 2021-03-12 15:58:10 punctuation_c